In [2]:
import pandas as pd
import os
import pathlib

In [3]:
datafile = '..\data\schedule.xlsx'

In [5]:
vac = pd.read_excel(datafile,sheet_name='Vacation')

In [6]:
days = pd.read_excel(datafile,sheet_name='Days')

In [7]:
vac_sched = days.merge(vac,on='Date')

In [21]:
#build list of all potential months call work time units (week or weekend)
#and assign this an index
potential_call = days.callshiftassignment.drop_duplicates()\
    .reset_index().drop('index',axis=1).reset_index()

In [22]:
vac_slots = vac_sched[['week','callshiftassignment','Person']].drop_duplicates()\
    .sort_values(by=['week','Person'])[['callshiftassignment','Person']].drop_duplicates()

In [23]:
vac_slots = vac_slots.merge(potential_call,on='callshiftassignment')

In [24]:
vac_slots 

,callshiftassignment,Person,index
0,weekend8,Jess,13
1,weekday8,Jess,14
2,weekend9,Jess,15
3,weekend10,Deenah,17
4,weekday10,Deenah,18
5,weekend11,Deenah,19
6,weekend11,Paul,19
7,weekday11,Deenah,20
8,weekday11,Paul,20
9,weekend12,Erin,21


In [25]:
potential_call

,index,callshiftassignment
0,0,weekday0
1,1,weekend1
2,2,weekday1
3,3,weekend2
4,4,weekday2
5,5,weekend3
6,6,weekday3
7,7,weekend4
8,8,weekday4
9,9,weekend5


In [216]:
#write the scheduled vacation weeks into a dict
#this shows which weeks/weekends people cannot work
vac_weeks = {}
for row in vac_slots.index:
    
    data = vac_slots.loc[row]
    if data[1] not in vac_weeks:
        vac_weeks[data[1]]= [data[0]]

    if data[1] in vac_weeks:
        if data[0] not in vac_weeks[data[1]]:
            vac_weeks[data[1]].append(data[0])

In [27]:
vac_weeks

{'Jess': [13, 14, 15],
 'Deenah': [17, 18, 19, 20],
 'Paul': [19, 20, 21],
 'Erin': [21, 22, 23]}

In [217]:
potential_call

,index,callshiftassignment
0,0,weekday0
1,1,weekend1
2,2,weekday1
3,3,weekend2
4,4,weekday2
5,5,weekend3
6,6,weekday3
7,7,weekend4
8,8,weekday4
9,9,weekend5


In [218]:
#determine which weeks/weekends people can work
#construct a dict of potential call weeks/weekends
not_vac_weeks = {}

for person in vac_weeks:
    for week in potential_call.callshiftassignment:

        #create first entry
        if person not in not_vac_weeks:
            if week not in vac_weeks[person]:
                not_vac_weeks[person] = [week]

        if person in not_vac_weeks:
            if week not in vac_weeks[person]:
                if week not in not_vac_weeks[person]:
                    not_vac_weeks[person].append(week)



In [219]:
not_vac_weeks['Deenah']

['weekday0',
 'weekend1',
 'weekday1',
 'weekend2',
 'weekday2',
 'weekend3',
 'weekday3',
 'weekend4',
 'weekday4',
 'weekend5',
 'weekday5',
 'weekend7',
 'weekday7',
 'weekend8',
 'weekday8',
 'weekend9',
 'weekday9',
 'weekend12',
 'weekday12',
 'weekend13']

In [220]:
vac_weeks['Deenah']

['weekend10', 'weekday10', 'weekend11', 'weekday11']

In [34]:
import pulp

In [221]:
residents = list(set(vac.Person))

In [222]:
shifts = list(potential_call.callshiftassignment)

In [223]:
x = pulp.LpVariable.dicts(
    'x',
    ((shift, resident)
        for resident in residents
        for shift in shifts),
    cat=pulp.LpBinary)

In [225]:
model = pulp.LpProblem('Schedule', pulp.LpMaximize)

#constraints for residents
for resident in residents:

    # this is the "no time-turners constraint"
     
    #minumum number of shifts 
    model.addConstraint(
        sum(x[shift, resident] for shift in shifts) >= 5,
        F'{6} shifts per resident {resident}')

    #remove vacation weeks
    for s in vac_weeks[resident]:
        model.addConstraint(
                sum(x[s, resident] for shift in shifts) == 0 ,
                '{} cannot work during {}'.format(resident,s))

for shift in shifts:
    
    #no duplication of shifts
    model.addConstraint(
        sum(x[shift, resident] for resident in residents) == 1,
        F'{1} resident per {shift}')


In [226]:
# pulp.LpSolverDefault.msg = 1
model.solve()
if pulp.LpStatus[model.status] != 'Optimal':
    raise ValueError(pulp.LpStatus[model.status])

data = [{
    'resident': resident,
    'shift':shift,
    'result': x[shift,resident].varValue}
 for resident in residents for shift in shifts]

result_raw = pd.DataFrame(data)
result_final = result_raw[result_raw.result > 0]
result_final

,resident,shift,result
0,Erin,weekday0,1.0
6,Erin,weekday3,1.0
8,Erin,weekday4,1.0
11,Erin,weekend7,1.0
16,Erin,weekday9,1.0
17,Erin,weekend10,1.0
25,Deenah,weekend1,1.0
28,Deenah,weekday2,1.0
31,Deenah,weekend4,1.0
34,Deenah,weekday5,1.0


In [227]:
result_final[result_final.resident == 'Deenah']

,resident,shift,result
25,Deenah,weekend1,1.0
28,Deenah,weekday2,1.0
31,Deenah,weekend4,1.0
34,Deenah,weekday5,1.0
37,Deenah,weekend8,1.0
38,Deenah,weekday8,1.0


In [213]:
result_final_out = result_final.merge(potential_call,left_on='shift',right_on='index')\
    .merge(days,on='callshiftassignment')

In [215]:
result_final_out.to_clipboard()

In [210]:
vac_weeks['Deenah']

[17, 18, 19, 20]